In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from src.models.loss import LossWrapper,Type2DirichletLoss
from src.data.data_loader import load_diwan_test,load_diwan_train,load_final,JerseyNumberDataset
from src.data.data_handling import split_dataset, balancer
from src.models.train_test import train, plot_history
from src.models.train_test import train, test, grid
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
chemin_model="results/weights/new_aug_20K_blur_0.3_dirichlet_thresh0.6.pth"
model_name = 'new_aug_20K_blur_0.3_dirichlet_thresh0.6'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = torch.load(chemin_model, map_location=device)
model.eval()
b = model

Using device: cuda


C:\Users\skouz\AppData\Local\Temp\ipykernel_8136\2604237465.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(chemin_model, map_location=device)


In [4]:
path1 = "data/diwan/test"
path2_1 = "data/diwan/train/seif_train_gt.json"
path2_2 = "data/diwan/train/skander_train_gt.json"
data1 = load_diwan_test(path1)
data2 = load_diwan_train([path2_1,path2_2])
data = data1 + data2

In [5]:
data_fine = balancer(data,max_0 = 0)
print("total finetune : ", len(data_fine))

total finetune :  3887


In [6]:
path_test = "data/final/images"
test_data = load_final(path_test)
test_ndhif = balancer(test_data,max_0 = 0)
print("total test : ", len(test_ndhif))

total test :  1028


In [7]:
train_data, valid_data = split_dataset(data_fine, split_ratio=0.95)
print(len(train_data),"train samples")
print(len(valid_data),"valid samples")

3692 train samples
195 valid samples


In [8]:
cut = "topbottom"
image_size = (224, 224)
train_dataset = JerseyNumberDataset(train_data, image_size=image_size,cut=cut,proof = True)
valid_dataset = JerseyNumberDataset(valid_data, image_size=image_size,cut=cut,proof = True)

In [9]:
# DataLoaders
batch_size = 64
workers = 1
train_loader = DataLoader(train_dataset,num_workers=workers, batch_size=batch_size, shuffle=True,pin_memory=True)
val_loader = DataLoader(valid_dataset,num_workers=workers, batch_size=batch_size, shuffle=False,pin_memory=True)

In [10]:
test_dataset = JerseyNumberDataset(test_ndhif, image_size=image_size,cut=cut)
test_loader = DataLoader(test_dataset,num_workers=workers, batch_size=batch_size, shuffle=True,pin_memory=True)

In [11]:
test_metrics = test(model, test_loader, device=device)

Mask accuracy(unc_threshold=0.5) : 1.0000

Accuracy:  0.5185

Precision: 0.4133

Recall:    0.3745

F1-score:  0.3629



In [12]:
num_epochs = 10
lr = 1e-5
loss_fn = LossWrapper(Type2DirichletLoss(num_classes=100))
optimizer = optim.Adam(model.parameters(), lr=lr)

In [13]:
history = train(model=model,train_loader=train_loader,val_loader=val_loader,
    optimizer=optimizer,loss_fn=loss_fn,device=device,num_epochs=num_epochs)


Epoch 1/10


Validation: 100%|██████████| 4/4 [00:03<00:00,  1.25it/s]


Train loss: 1.8600 | Train acc: 0.5888 | Val loss: 1.6786 | Val acc: 0.6000 | Time: 16.72s 

Epoch 2/10


Training:  40%|███▉      | 23/58 [00:07<00:10,  3.21it/s]


KeyboardInterrupt: 

In [ ]:
fig = plot_history(history)

In [ ]:
test_metrics = test(model, test_loader, device=device)

In [ ]:
image_grid = grid(model, test_loader, device=device)

In [ ]:
image_grid.savefig(f'results/finetuned_grids/ftd_{model_name}.png')
plt.close(fig)

In [ ]:
new_row = {
    "model_name": f"ftd_{model_name}",
    "acc": test_metrics["acc"],
    "precision": test_metrics["precision"],
    "recall": test_metrics["recall"],
    "f1": test_metrics["f1"]
}
csv_path = "tableau_final.csv"
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
else:
    df = pd.DataFrame(columns=["model_name", "acc", "precision", "recall", "f1"])

# === Ajouter la nouvelle ligne ===
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# === Sauvegarder le CSV mis à jour ===
df.to_csv(csv_path, index=False)

# === Afficher le tableau final ===
display(df)

In [ ]:
train_data1, valid_data1 = split_dataset(test_ndhif, split_ratio=0.9)
print(len(train_data1),"train samples")
print(len(valid_data1),"valid samples")

In [ ]:
train_dataset1 = JerseyNumberDataset(train_data1, image_size=image_size,cut=cut)
valid_dataset1 = JerseyNumberDataset(valid_data1, image_size=image_size,cut=cut)

In [ ]:
train_loader1 = DataLoader(train_dataset1,num_workers=workers, batch_size=batch_size, shuffle=True,pin_memory=True)
val_loader1 = DataLoader(valid_dataset1,num_workers=workers, batch_size=batch_size, shuffle=False,pin_memory=True)

In [ ]:
lr_test = 1e-6
optimizer_test = optim.Adam(model.parameters(), lr=lr_test)
history = train(model=model,train_loader=train_loader1,val_loader=val_loader1,
    optimizer=optimizer,loss_fn=loss_fn,device=device,num_epochs=num_epochs)

In [ ]:
torch.save(model, f"results/finetuned_models/ftd_{model_name}.pth")
